In [1]:
import altair as alt
import pandas as pd

# Create sample data
df = pd.DataFrame({
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'y': [2, 4, 5, 4, 6, 7, 8, 7, 9, 10]
})

# Create scatter plot with regression line
chart = alt.Chart(df).mark_point(opacity=0.7).encode(
    x='x',
    y='y'
) + alt.Chart(df).mark_line(color='red').encode(
    x='x',
    y=alt.Y('y', aggregate='mean')
)

chart.properties(title='Altair Regression Plot')

alt.LayerChart(...)

In [4]:
from math import log
log(5) / log(4)

1.160964047443681

# Notes

I want to compare convergence rates for Poisson and Pareto.

I'm going to assume a population mean of 15 minutes (TTR). 